# Objetivo del notebook

**Feature Engineering para Predicción Semanal de Bollería**

Este notebook tiene como objetivo preparar el dataset final que será utilizado para entrenar un modelo de predicción de ventas semanales en la categoría **Bollería**. Parte de los datos diarios ya validados y realiza todo el proceso de agregación, enriquecimiento, generación de variables temporales y preparación de los conjuntos de datos `X` e `y`.



🗂️ Entradas
- `validated_range_diario.parquet`: archivo diario con los datos corregidos, sin valores nulos, y con columnas de contexto como `is_weekend`, `is_easter`, `is_summer_peak`, etc.



⚙️ Proceso realizado

1. **Filtrado por familia "Bollería"** y selección de variables clave (`fecha`, `base_imponible`, `total`, etc.).
2. **Agregación semanal** usando el lunes como primer día de la semana.
3. **Asignación precisa de la variable `is_easter`** usando fechas exactas de Semana Santa (no solo la ISO week).
4. **Cálculo de variables contextuales** a nivel semanal como:
   - `is_summer_peak`: si hay algún día de verano "pico" en la semana.
   - `is_easter`: si hay algún día de Semana Santa en la semana.
5. **Generación de variables lag** (`lag_1`, `lag_2`, `lag_3`, `lag_4`, `lag_12`, `lag_24`, `lag_52`) sobre la variable `base_imponible` para capturar patrones temporales semanales y anuales.
6. **Eliminación de filas con valores nulos** introducidos por los lags.
7. **Selección de variables de entrada (X) y target (y)** para el entrenamiento del modelo:
   - **Features**: lags + variables contextuales (`is_summer_peak`, `is_easter`)
   - **Target**: `base_imponible`
8. **Exportación final del dataset** a `ts_df_bolleria_baseline.parquet` listo para el entrenamiento del modelo.



📤 Salidas
- `ts_df_bolleria_semanal.parquet`: serie semanal enriquecida (previa a aplicar lags).
- `ts_df_bolleria_baseline.parquet`: dataset final con lags y sin nulos, listo para modelar.



📌 Notas
- La lógica de `is_easter` se basa en las fechas exactas de Semana Santa, no en semanas ISO, para reflejar mejor el comportamiento real del negocio.
- Los lags se calculan sólo hacia el pasado (nunca con información futura).
- Este notebook no entrena modelos, solo **deja todo listo para el siguiente paso: entrenamiento**.





📆 Transformación de datos diarios a semanales y creación de variables estacionales

En esta celda se parte del dataset diario validado y se:

- Filtra la familia "Bollería".
- Agrega la información a nivel **semanal**, sumando la `base_imponible` y `total`.
- Añade columnas temporales (`year`, `week`) y de contexto (`is_summer_peak`, `is_easter`).
- Detecta con precisión si en esa semana hubo al menos un día de Semana Santa o verano pico.

El resultado es una serie semanal enriquecida que captura patrones estacionales relevantes.


# Agregación Semanal y Etiquetado de Eventos Especiales

📆 Transformación de datos diarios a semanales y creación de variables estacionales

En esta celda se parte del dataset diario validado y se:

- Filtra la familia "Bollería".
- Agrega la información a nivel **semanal**, sumando la `base_imponible` y `total`.
- Añade columnas temporales (`year`, `week`) y de contexto (`is_summer_peak`, `is_easter`).
- Detecta con precisión si en esa semana hubo al menos un día de Semana Santa o verano pico.

El resultado es una serie semanal enriquecida que captura patrones estacionales relevantes.


In [4]:
import pandas as pd
from dateutil.easter import easter
import os

# Importar path centralizado para los datos de entrada
try:
    from src.paths import VALIDATED_RANGE_FECHA_FAMILIA
except ImportError:
    import sys
    sys.path.append('..')
    from src.paths import VALIDATED_RANGE_FECHA_FAMILIA

# 1. Carga del diario corregido
df_diario = pd.read_parquet(str(VALIDATED_RANGE_FECHA_FAMILIA))
df_diario['fecha'] = pd.to_datetime(df_diario['fecha'])

# 1b. Filtrar solo fechas entre 2023-01-02 (lunes) y 2025-06-29 (domingo)
fecha_inicio = pd.to_datetime('2023-01-02')
fecha_fin = pd.to_datetime('2025-06-29')
df_diario = df_diario[(df_diario['fecha'] >= fecha_inicio) & (df_diario['fecha'] <= fecha_fin)]

# 2. Recalcular is_easter en el diario por si no estuviera:
iso = df_diario['fecha'].dt.isocalendar()
df_diario['year_iso'] = iso['year']
df_diario['week_iso'] = iso['week']

df_diario['is_easter'] = df_diario.apply(
    lambda r: int(r['week_iso'] == easter(r['year_iso']).isocalendar()[1]),
    axis=1
)

# 3. Agregar a nivel semanal (todas las familias) y contar días por semana
df_diario['semana_id'] = df_diario['year_iso'].astype(str) + '-' + df_diario['week_iso'].astype(str)
conteo_dias = df_diario.groupby(['year_iso','week_iso','familia'])['fecha'].count().reset_index(name='dias_semana')
df_semanal = (
    df_diario
      .groupby(['year_iso','week_iso','familia'], as_index=False)
      .agg({
         'base_imponible': 'sum',
         'is_summer_peak': 'max',
         'is_easter':      'max'
      })
    .merge(conteo_dias, on=['year_iso','week_iso','familia'])
)

# 3b. Filtrar solo semanas completas (7 días)
df_semanal = df_semanal[df_semanal['dias_semana'] == 7]



## Filtramos por categoría (bollería)

In [5]:
# 4. Filtrar solo BOLLERIA
df_bolleria_semanal = df_semanal.query("familia=='BOLLERIA'").copy()

# 5. Renombrar columnas y ordenar
df_bolleria_semanal.rename(columns={
    'year_iso':'year',
    'week_iso':'week'
}, inplace=True)
df_bolleria_semanal = df_bolleria_semanal.sort_values(['year','week']).reset_index(drop=True)

# 6. Guardar en processed usando path centralizado
from src.paths import TS_DF_BOLLERIA_SEMANAL
df_bolleria_semanal.to_parquet(str(TS_DF_BOLLERIA_SEMANAL), index=False)

print(f"Nuevo ts_df_bolleria_semanal guardado en: {TS_DF_BOLLERIA_SEMANAL}")
print("Comprobación is_easter por año:", df_bolleria_semanal.groupby('year')['is_easter'].sum().to_dict())
print(f"Semanas finales: {df_bolleria_semanal['year'].min()}-{df_bolleria_semanal['week'].min()} a {df_bolleria_semanal['year'].max()}-{df_bolleria_semanal['week'].max()}")


Nuevo ts_df_bolleria_semanal guardado en: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_semanal.parquet
Comprobación is_easter por año: {np.uint32(2023): 1, np.uint32(2024): 1, np.uint32(2025): 1}
Semanas finales: 2023-1 a 2025-51


In [6]:
import pandas as pd
from src.paths import TS_DF_BOLLERIA_SEMANAL

# Visualización de las primeras filas de la tabla semanal de bollería
df_bolleria_semanal = pd.read_parquet(str(TS_DF_BOLLERIA_SEMANAL))
df_bolleria_semanal.head()

,year,week,familia,base_imponible,is_summer_peak,is_easter,dias_semana
0,2023,1,BOLLERIA,825.11,0,0,7
1,2023,2,BOLLERIA,658.40,0,0,7
2,2023,3,BOLLERIA,741.40,0,0,7
3,2023,4,BOLLERIA,653.64,0,0,7
4,2023,5,BOLLERIA,680.46,0,0,7


# Generación de Lags y Dataset para Modelado

🧠 **Ingeniería de variables lag y preparación del dataset de entrenamiento**

Esta celda:

- Carga el dataset semanal enriquecido (`ts_df_bolleria_semanal.parquet`).
- Asegura el orden temporal por `year` y `week`.
- Crea **lags temporales** (`lag_1`, `lag_2`, ..., `lag_52`) sobre `base_imponible`.
- Elimina las filas con `NaN` que surgen por los lags.
- Define las variables predictoras (`X`) y la variable objetivo (`y`).
- Guarda el dataset final (`ts_df_bolleria_baseline.parquet`) listo para entrenamiento.

Esta fase cierra el proceso de **feature engineering** y deja los datos listos para modelar.


In [7]:
import pandas as pd
from src.paths import TS_DF_BOLLERIA_SEMANAL, TS_DF_BOLLERIA_BASELINE

# 1. Carga de la serie semanal procesada
df = pd.read_parquet(str(TS_DF_BOLLERIA_SEMANAL))

# 2. Asegurar orden temporal
df = df.sort_values(['year','week']).reset_index(drop=True)

# 3. Definición de los lags a crear
selected_lags = [1, 2, 3, 4, 12, 24, 52]

# 4. Generar las columnas lag_X
for lag in selected_lags:
    df[f'lag_{lag}'] = df['base_imponible'].shift(lag)

# 5. Eliminar filas con NaN (introducidas por los lags)
df_features = df.dropna().reset_index(drop=True)

# 6. Selección de las columnas para el baseline
feature_cols = [f'lag_{lag}' for lag in selected_lags] + ['is_summer_peak','is_easter']
target_col   = 'base_imponible'

X = df_features[feature_cols]
y = df_features[target_col]

print("Shape X:", X.shape)
print("Features ejemplo:")
print(X.head())

# 7. Guardar este dataset listo para modelado usando path centralizado
df_features.to_parquet(str(TS_DF_BOLLERIA_BASELINE), index=False)
print("Dataset de features guardado en:", TS_DF_BOLLERIA_BASELINE)


Shape X: (74, 9)
Features ejemplo:
    lag_1   lag_2   lag_3   lag_4      lag_12   lag_24  lag_52  \
0  572.51  534.79  563.18  806.54  912.130000  1745.97  825.11   
1  597.65  572.51  534.79  563.18  750.990000  1681.41  658.40   
2  680.30  597.65  572.51  534.79  821.840000  1753.02  741.40   
3  603.99  680.30  597.65  572.51  895.391469  1835.18  653.64   
4  600.14  603.99  680.30  597.65  743.259393  2127.71  680.46   

   is_summer_peak  is_easter  
0               0          0  
1               0          0  
2               0          0  
3               0          0  
4               0          0  
Dataset de features guardado en: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_baseline.parquet
